<a href="https://colab.research.google.com/github/marcusflygar1-hash/project_testing/blob/main/Traffic_project_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the neccesary libraries for the project.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import auc,roc_auc_score,roc_curve, accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, r2_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

Nya features att addera:

Veckodagar, månad & ?


Gör modellen mer robust, genom att normalisera portal datan.

GridSearchCV? .. tar lång tid dock...



# Data prepping

In [ ]:
tds = pd.read_csv('training_dataset.csv', sep=';')
ev_ds = pd.read_csv('evaluation_dataset.csv',sep=';')
f_ev_ds = pd.read_csv('final_evaluation_dataset.csv', sep=';')


In [ ]:
print('\n training dataset')
print(tds.head())
print('\n evaluation dataset------------')
print(ev_ds.head())
print('\n final evaluation dataset --------')
print(f_ev_ds.head())

In [ ]:
print('Training dataset')
print(tds.isna().sum())
print('\n eval dataset')
print(ev_ds.isna().sum())
print('\n final eval dataset')
print(f_ev_ds.isna().sum())

In [ ]:
#drop all NaN values.
f_ev_ds_noNAN = f_ev_ds.dropna()
tds_noNAN = tds.dropna()
ev_ds_noNAN = ev_ds.dropna()

In [ ]:
tds_noNAN.head(10)

In [ ]:
#Filter all of the datasets so we only have data from the correct portal / sensor group
#This will ensure we can get more reliable data as the predicitons just do not work when they are all together.

portal_of_interest = "E4S 56,780"
tds_noNAN = tds_noNAN[tds_noNAN['PORTAL'] == portal_of_interest]
ev_ds_noNAN = ev_ds_noNAN[ev_ds_noNAN['PORTAL'] == portal_of_interest]
f_ev_ds_noNAN = f_ev_ds_noNAN[f_ev_ds_noNAN['PORTAL'] == portal_of_interest]
print(f"Training rows for {portal_of_interest}: {len(tds_noNAN)}")
print(f"Evaluation rows for {portal_of_interest}: {len(ev_ds_noNAN)}")

In [ ]:
print(f'Org dataset amount:{len(tds)}')
print(f'Org dataset amount:{len(ev_ds)}')
print(f'Org dataset amount:{len(f_ev_ds)}')

print(f'Dropped Dataset amount:{len(tds_noNAN)}')
print(f'Dropped Dataset amount:{len(ev_ds_noNAN)}')
print(f'Dropped Dataset amount:{len(f_ev_ds_noNAN)}')

# Forts.

In [ ]:
#define what is congestion and what is not.
# congestion_flow = tds[tds['FLOW'] < 100]
# congestion_speed = tds[tds['SPEED_MS_AVG'] < 10]

ev_ds_noNAN['datetime'] = pd.to_datetime(ev_ds_noNAN['Date'].astype(str) + ' ' + ev_ds_noNAN['Time'])

ev_ds_offpeak = ev_ds_noNAN[(ev_ds_noNAN['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                            (ev_ds_noNAN['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]
ev_ds_onpeak = ev_ds_noNAN[(ev_ds_noNAN['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
                           (ev_ds_noNAN['datetime'].dt.time <= pd.to_datetime("08:30:00").time())]

ev_ds_offpeak = ev_ds_offpeak[(ev_ds_offpeak['FLOW'] > 0) &
                              (ev_ds_offpeak['SPEED_MS_AVG'].between(5, 50))]
ev_ds_onpeak = ev_ds_onpeak[(ev_ds_onpeak['FLOW'] > 0) &
                            (ev_ds_onpeak['SPEED_MS_AVG'].between(5, 50))]


In [ ]:
# converting the date and time columns int o just one "datetime" coluinm
tds_noNAN['datetime'] = pd.to_datetime(tds_noNAN['Date'].astype(str) + ' ' + tds_noNAN['Time'])

# filtering the dataset to only include the data within hours 04:00 and 06:00, this to get an accurate description of free flow speed.
tds_offpeak = tds_noNAN[(tds_noNAN['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                (tds_noNAN['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]

tds_onpeak= tds_noNAN[(tds_noNAN['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
                  (tds_noNAN['datetime'].dt.time <= pd.to_datetime("08:30:00").time())]
#Ensuring we only get correctly read speeds. E.g Removing any negative speeds and random slow drivers etc
#that do not actually depict the actual free flowspeed e.g people speeding and drinving super slow...
tds_offpeak = tds_offpeak[(tds_offpeak['FLOW'] > 0) &
                        (tds_offpeak['SPEED_MS_AVG'].between(2, 50))]
tds_onpeak = tds_onpeak[(tds_onpeak['FLOW'] > 0) &
                        (tds_onpeak['SPEED_MS_AVG'].between(2, 50))]

In [ ]:
#defining what congestion is for off peak hours
ffs_kmh = 85 #Through basic theory, a 4 lane road with a speed limit of 70 km/h, would have the free flow speed around 85 km/h
ffs_ms = ffs_kmh * 0.277778 # Converting km/h to m/s
cong_th = (ffs_kmh * 0.7) # We use mild congestion as a staple, as otherwise the dataset for congested times is just to small to work with.
print(f"Free flow speed in [km/h]: {ffs_kmh}")
print(f"Free flow speed in [m/s]: {ffs_ms}")
print(f"Congestion Speed Threshold [km/h]: {cong_th}")

#hard coding cong as 60 [km/h] for simplicty
cong_th_2 = 60

In [ ]:
#defining what congestion is for on peak hours
ffs_kmh_2 = 85 #Through basic theory, a 4 lane road with a speed limit of 70 km/h, would have the free flow speed around 85 km/h
ffs_ms_2 = ffs_kmh_2 * 0.277778 # Converting km/h to m/s
cong_th_2 = (ffs_kmh_2 * 0.7) # We use mild congestion as a staple, as otherwise the dataset for congested times is just to small to work with.
print(f"Free flow speed in [km/h]: {ffs_kmh_2}")
print(f"Free flow speed in [m/s]: {ffs_ms_2}")
print(f"Congestion Speed Threshold [km/h]: {cong_th_2}")

#hard coding cong as 61 [km/h] for simplicty and to catch those going around 60km/h for a fairer comparison
cong_th_2 = 61

In [ ]:
#Distribution of speeds during on peak hours
plt.figure(figsize=(10,6))
plt.hist(tds_onpeak['SPEED_MS_AVG']*3.6, bins=40, color='skyblue', edgecolor='black', alpha=0.7)

plt.axvline(ffs_kmh_2, color='red', linestyle='--', linewidth=2, label=f'85th percentile (FFS = {ffs_kmh_2:.2f} km/h)')
plt.axvline(cong_th_2, color='green', linestyle='--', linewidth=2, label=f'Congestion threshold (70% of FFS = {cong_th_2:.2f} km/h)')
plt.xlabel("Speed (km/h)")
plt.ylabel("Frequency")
plt.title("Distribution of Speeds During On-Peak (07:30–08:30)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
#dist of speeds during off peak hours.

# Plot histogram
plt.figure(figsize=(10,6))
plt.hist(tds_offpeak['SPEED_MS_AVG']*3.6, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(ffs_kmh, color='red', linestyle='--', linewidth=2, label=f'85th percentile (FFS = {ffs_kmh:.2f} km/h)')
plt.axvline(cong_th, color='green', linestyle='--', linewidth=2, label=f'Congestion threshold (70% of FFS = {cong_th_2:.2f} km/h)')



plt.xlabel("Speed (km/h)")
plt.ylabel("Frequency")
plt.title("Distribution of Speeds During Off-Peak (04:00–06:00)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
print(cong_th_2)
tds_onpeak.info()

In [ ]:
#Creat our rolling and shifting.
tds_onpeak['pred15'] = tds_onpeak['SPEED_MS_AVG'].shift(-1).rolling(15).mean()
tds_onpeak['congestion'] = (tds_onpeak['pred15']*3.6 <= cong_th_2).astype(int)

In [ ]:
tds_onpeak["speed_norm"] = (
    tds_onpeak.groupby("DP_ID")["SPEED_MS_AVG"]
    .transform(lambda x: (x - x.mean()) / x.std())
)

ev_ds_onpeak["speed_norm"] = (
    ev_ds_onpeak.groupby("DP_ID")["SPEED_MS_AVG"]
    .transform(lambda x: (x - x.mean()) / x.std())
)

In [ ]:
def make_features_targets(df, horizon=15, cong_th_2=50):
    df = df.copy().sort_values("datetime")

    # rolling(horizon).mean() looks backward, so there fore shift looks *backwards* by 'horizon' to make it look intothe future
    df[f'future_speed_{horizon}m'] = (
        df['SPEED_MS_AVG']
        .rolling(window=horizon, closed='right')
        .mean()
        .shift(-horizon) * 3.6
    )

    # sets a target where if future speed is less than congestion speed.
    df['future_congestion'] = (df[f'future_speed_{horizon}m'] <= cong_th_2).astype(int)

    # historical 15 minutes of data
    df['past15_flow_mean']  = df['FLOW'].rolling(15).mean()
    df['past15_flow_std']   = df['FLOW'].rolling(15).std()
    df['past15_flow_min']   = df['FLOW'].rolling(15).min()
    df['past15_flow_max']   = df['FLOW'].rolling(15).max()
    df['past15_flow_slope'] = df['FLOW'] - df['FLOW'].shift(15)

    df['past15_speed_mean']  = df['SPEED_MS_AVG'].rolling(15).mean()
    df['past15_speed_std']   = df['SPEED_MS_AVG'].rolling(15).std()
    df['past15_speed_min']   = df['SPEED_MS_AVG'].rolling(15).min()
    df['past15_speed_max']   = df['SPEED_MS_AVG'].rolling(15).max()
    df['past15_speed_slope'] = df['SPEED_MS_AVG'] - df['SPEED_MS_AVG'].shift(15)
    df['past15_speed_norm_mean'] = df['speed_norm'].rolling(15).mean()
    df['past15_speed_norm_std']  = df['speed_norm'].rolling(15).std()


    #  trend features
    df["flow_delta_5_15"]  = df["FLOW"].rolling(5).mean() - df["FLOW"].rolling(15).mean()
    df["speed_delta_5_15"] = df["SPEED_MS_AVG"].rolling(5).mean() - df["SPEED_MS_AVG"].rolling(15).mean()
    df["speed_diff_1"] = df["SPEED_MS_AVG"].diff(1)
    df["flow_diff_1"]  = df["FLOW"].diff(1)
    df["speed_range_15"] = df["SPEED_MS_AVG"].rolling(15).max() - df["SPEED_MS_AVG"].rolling(15).min()
    df["speed_ratio_15"] = df["SPEED_MS_AVG"].rolling(15).min() / df["SPEED_MS_AVG"].rolling(15).max()


    # time related features.
    df["minute_of_day"] = df["datetime"].dt.hour * 60 + df["datetime"].dt.minute
    df["tod_sin"] = np.sin(2 * np.pi * df["minute_of_day"] / 1440)
    df["tod_cos"] = np.cos(2 * np.pi * df["minute_of_day"] / 1440)
    df["hour"] = df["datetime"].dt.hour
    df["dayofweek"] = df["datetime"].dt.dayofweek
    df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)


    # full feature list
    features = [
        'past15_flow_mean','past15_flow_std','past15_flow_min','past15_flow_max','past15_flow_slope',
        'past15_speed_mean','past15_speed_std','past15_speed_min','past15_speed_max','past15_speed_slope',
        'tod_sin','tod_cos'
        ]
    #target "variable"
    target = 'future_congestion'

    # dropping NaN values if they have appeared due to rolling and shift.
    df = df.dropna(subset=features + [target])

    return df, features, target


In [ ]:
tds_fixad, features, target = make_features_targets(tds_onpeak, horizon=15, cong_th_2=cong_th_2)

# 2. Create evaluation data (for 2022)
ev_fixad, _, _ = make_features_targets(tds_onpeak, horizon=15)


## Dela upp i train test splits.

In [ ]:
#Split into train / test sets
X = tds_fixad[features].values
y = tds_fixad[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
#Scaling the datat.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Test data models

In [ ]:
from sklearn.linear_model import LogisticRegression

#perform linear regression model.
lr = LogisticRegression(random_state = 42, max_iter = 1000, class_weight='balanced')
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_prob_lr = lr.predict_proba(X_test_scaled)[:, 1]

print("Logistic Regression Dataset")
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, digits=3))
print(f" The roc_auc score is: {roc_auc_score(y_test, y_prob_lr)}")

In [ ]:
print("Class distribution in y_train:")
print(pd.Series(y_train).value_counts())

In [ ]:
print("Class distribution in tds_fixad['future_congestion']:")
print(tds_fixad['future_congestion'].value_counts())

In [ ]:
from xgboost import XGBClassifier
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)
xgb_clf = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)

xgb_clf.fit(X_train_scaled, y_train)

# Predict.
y_pred_xgb = xgb_clf.predict(X_test_scaled)
y_prob_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]


# stats
print("XGBoost Training dataset")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, digits=3))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    class_weight='balanced',  # handle imbalance
    random_state=42,
    n_jobs=-1
)

rf_clf.fit(X_train_scaled, y_train)
y_pred_rf = rf_clf.predict(X_test_scaled)
#stats
print("Random Forest Training dataset")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))

In [ ]:
print("Logistic Regression Dataset")
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, digits=3))
print("\n")

print("XGBoost Training dataset")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, digits=3))
print("\n")
print("Random Forest Training dataset")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))



In [ ]:
#create empty list to store results
results = []

#for loop that adds results to the list "results"
for name, pred in [
    ("Logistic Regression", y_pred_lr),
    ("Random Forest", y_pred_rf),
    ("XGBoost", y_pred_xgb)
]:
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, pred),
        "Precision": precision_score(y_test, pred),
        "Recall": recall_score(y_test, pred),
        "F1": f1_score(y_test, pred)
    })

results_df = pd.DataFrame(results)
#print the results.
print(results_df)


# Evaluation data models

In [ ]:
ev_fixad, _, _ = make_features_targets(ev_ds_onpeak, horizon=15, cong_th_2=cong_th_2)

In [ ]:
X_eval = ev_fixad[features].values
y_eval = ev_fixad[target].values
X_eval_scaled = scaler.transform(X_eval)

In [ ]:
#loop through models on evaluation dataset.
for name, model in [
    ("Logistic Regression", lr),
    ("Random Forest", rf_clf),
    ("XGBoost", xgb_clf)
]:
    y_pred_eval = model.predict(X_eval_scaled)
    print(f"\n{name} (Evaluation 2022 dataset):")
    print(confusion_matrix(y_eval, y_pred_eval))
    print(classification_report(y_eval, y_pred_eval, digits=3))